# Did you like the movie? - Cleaning stage

## Import Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

## Load the dataset

In [2]:
#I load the data set into the team4 variable, I specify that the data is separated by ; and that the index column will be Title. 
#I also speecified that the not available values are NA 
#I don't believe this last step was necessary but I did it anyways...
team4 = pd.read_csv('Catalogo_peliculas_22.csv',sep=';',index_col='Title',na_values='NA')


## Exploring the dataset

In [3]:
#We first verify that the data was importet correctly 
#As we can see it is correct ant the Title column is considered as the index, thats why it is a little lower 
team4.head()

,Year,Genre,Duration,Budget,Revenue,1,2,3,4,5,...,16,17,18,19,20,IMBD,Rotten_Tomatoes,Metacritic,Letterboxd,Personal
Title,,,,,,,,,,,,,,,,,,,,,
The Shawshank Redemption,1994,Drama,142,"28.341.469,00",2343110,90,75,86,61,74,...,87,93,70,76,62,93,99,98.0,84,80.0
The Godfather,1972,Crime,175,"134.966.411,00",1620367,78,98,73,71,85,...,85,60,98,76,96,92,80,97.0,91,100.0
The Dark Knight,2008,Action,152,"534.858.444,00",2303232,97,98,66,82,82,...,69,76,97,64,74,90,95,74.0,70,84.0
The Godfather Part II,1974,Crime,202,"57.300.000,00",1129952,72,78,70,68,95,...,63,87,61,83,75,90,82,75.0,91,90.0
12 Angry Men,1957,Crime,96,"4.360.000,00",689845,90,76,68,96,79,...,71,89,89,90,67,90,81,80.0,86,96.0


## Cleaning columns

Refer to the notebook `DB_assembling.ipynb` for cleaning column names for each team.

In [4]:
#We first look at all the columsn availble, just to make sure that we have the correct number of colums 
for df in [team4]: print(df.columns)


Index(['Year', 'Genre', 'Duration', 'Budget', 'Revenue', '1', '2', '3', '4',
       '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17',
       '18', '19', '20', 'IMBD ', 'Rotten_Tomatoes', 'Metacritic',
       'Letterboxd', 'Personal'],
      dtype='object')


## Cleaning data

We will follow the guidelines stated below for cleaning the data for each column:
- `Year`: This column will be left unchanged. Data type can be `str` or `int`.
- `Duration`: Data type should be `int`. Remove any indication to a unit of measurement.
- `Genre`: Data type should be `str`. We will capitalize the data, and if there is more than one genre, we will keep the first one and remove the res.
- `Budget`: Data type can be `int` or `float`. To that end, we will remove dollar signs as well as the thousands separator. We will scale the data so the unit of measurment is millions of dollars.
- `Revenue`: Same treatment as that of `Budget`.
- Rating columns: Data type can be `int` or `float`. Change scale to [0,100].

Data types are correct for all columns except `Revenue` and `Critic_3`.

### Team 4

In [5]:
#We first check the type of data that we have in the dataset. As we can see, most of our data is an object. We have to change this into the corresponding data type 
team4.info()
#We also print the shape to just to make sure that we only have 200 rows 
#sometimes the empty rows are also imported 
team4.shape

<class 'pandas.core.frame.DataFrame'>
Index: 200 entries, The Shawshank Redemption to Everything Everywhere All at Once
Data columns (total 30 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Year             200 non-null    int64  
 1   Genre            200 non-null    object 
 2   Duration         200 non-null    int64  
 3   Budget           195 non-null    object 
 4   Revenue          198 non-null    object 
 5   1                153 non-null    object 
 6   2                158 non-null    object 
 7   3                153 non-null    object 
 8   4                145 non-null    object 
 9   5                155 non-null    object 
 10  6                149 non-null    object 
 11  7                161 non-null    object 
 12  8                158 non-null    object 
 13  9                162 non-null    object 
 14  10               158 non-null    object 
 15  11               164 non-null    object 
 16  12            

(200, 30)

Data types are correct for all columns except `Duration`, `Budget`, `Revenue`, and `Fan_14`.

print(np.array(team4['Duration']))

In [6]:
#We print duration to make sure that it doesnt contain any strings ( we know it doesn't but just in case)
print(np.array(team4['Duration']))

[142 175 152 202  96 201 154 195 148 139 178 142 161 179 136 146 124 133
 160 132 153 169 130 125 169 189 116 127 118 121 133 207 130 122 106 112
 130 151 150 155 119 106 110  88 137 155  89 116 165 109 152  92 120 139
 141 124  98 148  95 132 111 180 140 149 164 120 137 181 121 165 122 123
 143 115  93  86 180 178 105 117 127 119 130 130 163 113 118 123 132 134
 118 134 124 121 147 161 176 156 144 124 103 207  99  93 129 106 145 143
 117 137 155 118 142 170 122 136 143 120  89  99 132  94 218 105 158 100
 117 126 102 159  95 130 106 143 115  91 143 132 103 100  94 137 169  99
 153  68 137  84 115 136 120  72  14 149 100  93 119 157 106 132 118 120
 125 102 147 135 134 127  94  91  75 115 183 129 156 122 140 133 122 144
 107 112 138  98 122 191 140 122 120 120 117 157 117 113 115 164 162 112
 109 140]


In [7]:
#I printed the budget and realise that the notation was different to the one expected 
# The dot is used as commas and the commas are used as dot 
#Before changing it into a float we have to fix this 
print(np.array(team4['Budget']))
#print(np.array(team4['Revenue']))

['28.341.469,00' '134.966.411,00' '534.858.444,00' '57.300.000,00'
 '4.360.000,00' '377.845.905,00' '107.928.762,00' '96.898.818,00'
 '292.576.195,00' '37.030.102,00' '315.544.750,00' '330.252.182,00'
 '6.100.000,00' '342.551.365,00' '171.479.930,00' '46.836.394,00'
 '290.475.067,00' '112.000.000,00' '390.305.102,00' '53.367.844,00' nan
 '188.020.017,00' '7.563.397,00' '10.055.859,00' '216.540.909,00'
 '136.801.374,00' '57.598.247,00' '100.125.643,00' '130.742.922,00'
 '322.740.140,00' '118.552.281,00' '269.061,00' nan '335.451.311,00'
 '13.092.000,00' '13.182.281,00' '53.089.891,00' '132.384.315,00'
 '32.572.577,00' '187.705.427,00' '6.719.864,00' '23.341.568,00'
 '19.501.238,00' '422.783.777,00' '204.843.350,00' '11.990.401,00' nan
 '210.609.762,00' '5.321.508,00' '522093' '97600000' '115000000'
 '150000000' '40000000' '52000000' '7000000' '165000000' '200000000'
 '175000000' '54000000' '150000000' '100000000' '150000000' '321000000'
 '250000000' '960000' '97000000' '356000000' '1750

In [8]:
#I realised the same happened with the Budget
#We also need to fix it
print(team4['Budget'])


Title
The Shawshank Redemption              28.341.469,00
The Godfather                        134.966.411,00
The Dark Knight                      534.858.444,00
The Godfather Part II                 57.300.000,00
12 Angry Men                           4.360.000,00
                                          ...      
Blade Runnner 2049                   150.000.000,00
Avatar                               237.000.000,00
Scott Pilgrim vs the world            85.000.000,00
Psycho                                   806.947,00
Everything Everywhere All at Once     25.000.000,00
Name: Budget, Length: 200, dtype: object


In [9]:
#I first took away the dots from both columns to keep the numbers and the  commas 
team4['Revenue']=team4['Revenue'].str.replace('.', '')
team4['Budget']=team4['Budget'].str.replace('.', '')
#I printed the column just to verify 
print (team4['Revenue'])


Title
The Shawshank Redemption                   2343110
The Godfather                              1620367
The Dark Knight                            2303232
The Godfather Part II                      1129952
12 Angry Men                                689845
                                         ...      
Blade Runnner 2049                    260500000,00
Avatar                               2924000000,00
Scott Pilgrim vs the world             48100000,00
Psycho                                 50000000,00
Everything Everywhere All at Once     143000000,00
Name: Revenue, Length: 200, dtype: object


In [10]:
team4.info()


<class 'pandas.core.frame.DataFrame'>
Index: 200 entries, The Shawshank Redemption to Everything Everywhere All at Once
Data columns (total 30 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Year             200 non-null    int64  
 1   Genre            200 non-null    object 
 2   Duration         200 non-null    int64  
 3   Budget           195 non-null    object 
 4   Revenue          198 non-null    object 
 5   1                153 non-null    object 
 6   2                158 non-null    object 
 7   3                153 non-null    object 
 8   4                145 non-null    object 
 9   5                155 non-null    object 
 10  6                149 non-null    object 
 11  7                161 non-null    object 
 12  8                158 non-null    object 
 13  9                162 non-null    object 
 14  10               158 non-null    object 
 15  11               164 non-null    object 
 16  12            

In [11]:
#AHORA HAGO LAS COMAS PUNTOS PARA QUE YA SEA LA NOTACION NORMAL DE SIEMPRE Y CONVIEROT A FLOTANTE 
#Now I change the commas into dots to be able to change the columns into floats
#THIS HAS TO BE DONE IN THIS ORDER, OTHERWISE IT WONT WORK
#Now that i'm doing the comments it makes sense but it took me a few minutes to realise it 
team4['Revenue']=team4['Revenue'].str.replace(',', '.')
team4['Revenue']=team4['Revenue'].astype(float)
team4['Budget']=team4['Budget'].str.replace(',', '.')
team4['Budget']=team4['Budget'].astype(float)
team4.info()


<class 'pandas.core.frame.DataFrame'>
Index: 200 entries, The Shawshank Redemption to Everything Everywhere All at Once
Data columns (total 30 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Year             200 non-null    int64  
 1   Genre            200 non-null    object 
 2   Duration         200 non-null    int64  
 3   Budget           195 non-null    float64
 4   Revenue          198 non-null    float64
 5   1                153 non-null    object 
 6   2                158 non-null    object 
 7   3                153 non-null    object 
 8   4                145 non-null    object 
 9   5                155 non-null    object 
 10  6                149 non-null    object 
 11  7                161 non-null    object 
 12  8                158 non-null    object 
 13  9                162 non-null    object 
 14  10               158 non-null    object 
 15  11               164 non-null    object 
 16  12            

In [12]:
#Check if it worked, it did 
team4.info()

<class 'pandas.core.frame.DataFrame'>
Index: 200 entries, The Shawshank Redemption to Everything Everywhere All at Once
Data columns (total 30 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Year             200 non-null    int64  
 1   Genre            200 non-null    object 
 2   Duration         200 non-null    int64  
 3   Budget           195 non-null    float64
 4   Revenue          198 non-null    float64
 5   1                153 non-null    object 
 6   2                158 non-null    object 
 7   3                153 non-null    object 
 8   4                145 non-null    object 
 9   5                155 non-null    object 
 10  6                149 non-null    object 
 11  7                161 non-null    object 
 12  8                158 non-null    object 
 13  9                162 non-null    object 
 14  10               158 non-null    object 
 15  11               164 non-null    object 
 16  12            

In [13]:
#Our data is in dollars, we need to change it into millions of dollars so we just divide and check again
team4['Budget']=team4['Budget']/1000000
team4['Revenue']=team4['Revenue']/1000000
print(np.array(team4['Budget']))
print(np.array(team4['Revenue']))

[2.83414690e+01 1.34966411e+02 5.34858444e+02 5.73000000e+01
 4.36000000e+00 3.77845905e+02 1.07928762e+02 9.68988180e+01
 2.92576195e+02 3.70301020e+01 3.15544750e+02 3.30252182e+02
 6.10000000e+00 3.42551365e+02 1.71479930e+02 4.68363940e+01
 2.90475067e+02 1.12000000e+02 3.90305102e+02 5.33678440e+01
            nan 1.88020017e+02 7.56339700e+00 1.00558590e+01
 2.16540909e+02 1.36801374e+02 5.75982470e+01 1.00125643e+02
 1.30742922e+02 3.22740140e+02 1.18552281e+02 2.69061000e-01
            nan 3.35451311e+02 1.30920000e+01 1.31822810e+01
 5.30898910e+01 1.32384315e+02 3.25725770e+01 1.87705427e+02
 6.71986400e+00 2.33415680e+01 1.95012380e+01 4.22783777e+02
 2.04843350e+02 1.19904010e+01            nan 2.10609762e+02
 5.32150800e+00 5.22093000e-01 9.76000000e+01 1.15000000e+02
 1.50000000e+02 4.00000000e+01 5.20000000e+01 7.00000000e+00
 1.65000000e+02 2.00000000e+02 1.75000000e+02 5.40000000e+01
 1.50000000e+02 1.00000000e+02 1.50000000e+02 3.21000000e+02
 2.50000000e+02 9.600000

In [14]:
#We have to change the columns names into fan_1, fan_2, etc. 
for df in [team4]: print(df.columns)


Index(['Year', 'Genre', 'Duration', 'Budget', 'Revenue', '1', '2', '3', '4',
       '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17',
       '18', '19', '20', 'IMBD ', 'Rotten_Tomatoes', 'Metacritic',
       'Letterboxd', 'Personal'],
      dtype='object')


In [15]:
#If the name of the column is a digit, it will change it 
rename_dict = {col: f'Fan_{col}' for col in team4.columns if col.isdigit()}
team4 = team4.rename(columns=rename_dict)
print(team4.columns)

Index(['Year', 'Genre', 'Duration', 'Budget', 'Revenue', 'Fan_1', 'Fan_2',
       'Fan_3', 'Fan_4', 'Fan_5', 'Fan_6', 'Fan_7', 'Fan_8', 'Fan_9', 'Fan_10',
       'Fan_11', 'Fan_12', 'Fan_13', 'Fan_14', 'Fan_15', 'Fan_16', 'Fan_17',
       'Fan_18', 'Fan_19', 'Fan_20', 'IMBD ', 'Rotten_Tomatoes', 'Metacritic',
       'Letterboxd', 'Personal'],
      dtype='object')


In [16]:
#As we can see in this column there are some values in the scale of 1- 10 
print(np.array(team4['Fan_14']))
team4.info()


['99' '60' '92' '83' '86' '78' '67' '99' '82' '86' '75' '76' '66' '88'
 '80' '67' '94' '85' '74' '61' '73' '67' '91' '92' '70' '90' '70' '71'
 '91' '69' '68' '72' '77' '73' '79' '61' '73' '86' '69' '95' '94' '71'
 '82' '73' '89' '66' '77' '91' '87' '75' '7' '7' nan '8' '9' '7' '7' '7,5'
 nan nan '8' nan '8' nan '8' '9,5' '6,5' '8' '7' nan '7' '7' '6' '7' '8'
 '6' '8,5' '7,5' '8' 'NS' '7' '7' '7' nan '8' '6' nan '9' '7,5' nan nan
 '8' '7,5' '8,5' '8' '8,5' nan '8' '7' nan '37' '22' '87' '52' '47' '67'
 '94' '94' '22' '27' '57' '70' '65' '79' '87' '17' '45' '78' '94' '73'
 '57' '32' '47' '97' '99' nan '37' nan '27' '72' '12' '17' '43' nan '92'
 '37' '77' '82' '70' '60' '37' '27' '47' '86' '87' nan '97' '94' '76' '36'
 nan nan '97' '88' '91' nan '97' nan '86' '88' '90' '94' nan nan nan nan
 '84' nan nan '66' '85' nan nan '98' nan '68' '76' nan '70' '99' nan nan
 nan nan '65' '73' '100' '69' '99' '89' '69' '74' '82' nan '91' '94' '97'
 nan nan nan]
<class 'pandas.core.frame.DataFrame'>
Ind

In [17]:
#I printed the columns just to make sure tht I only take the fans columns 
print (team4.columns[5:25])
print (np.array(team4['Fan_14']))

Index(['Fan_1', 'Fan_2', 'Fan_3', 'Fan_4', 'Fan_5', 'Fan_6', 'Fan_7', 'Fan_8',
       'Fan_9', 'Fan_10', 'Fan_11', 'Fan_12', 'Fan_13', 'Fan_14', 'Fan_15',
       'Fan_16', 'Fan_17', 'Fan_18', 'Fan_19', 'Fan_20'],
      dtype='object')
['99' '60' '92' '83' '86' '78' '67' '99' '82' '86' '75' '76' '66' '88'
 '80' '67' '94' '85' '74' '61' '73' '67' '91' '92' '70' '90' '70' '71'
 '91' '69' '68' '72' '77' '73' '79' '61' '73' '86' '69' '95' '94' '71'
 '82' '73' '89' '66' '77' '91' '87' '75' '7' '7' nan '8' '9' '7' '7' '7,5'
 nan nan '8' nan '8' nan '8' '9,5' '6,5' '8' '7' nan '7' '7' '6' '7' '8'
 '6' '8,5' '7,5' '8' 'NS' '7' '7' '7' nan '8' '6' nan '9' '7,5' nan nan
 '8' '7,5' '8,5' '8' '8,5' nan '8' '7' nan '37' '22' '87' '52' '47' '67'
 '94' '94' '22' '27' '57' '70' '65' '79' '87' '17' '45' '78' '94' '73'
 '57' '32' '47' '97' '99' nan '37' nan '27' '72' '12' '17' '43' nan '92'
 '37' '77' '82' '70' '60' '37' '27' '47' '86' '87' nan '97' '94' '76' '36'
 nan nan '97' '88' '91' nan '97' nan '86

In [18]:
#I choose the columns that I want to change 

columns_range = team4.columns[5:25]  
#On each column I replace the NA, Ns and Na (With a space)values 
#I also convert the type to a float 

for col in columns_range:
   
    team4[col] = team4[col].replace(['NA', 'NS','NA '], np.nan)
   
    team4[col] = team4[col].str.replace(',', '')
    
    team4[col] = team4[col].astype(float)


In [19]:
team4.info()

<class 'pandas.core.frame.DataFrame'>
Index: 200 entries, The Shawshank Redemption to Everything Everywhere All at Once
Data columns (total 30 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Year             200 non-null    int64  
 1   Genre            200 non-null    object 
 2   Duration         200 non-null    int64  
 3   Budget           195 non-null    float64
 4   Revenue          198 non-null    float64
 5   Fan_1            153 non-null    float64
 6   Fan_2            158 non-null    float64
 7   Fan_3            153 non-null    float64
 8   Fan_4            145 non-null    float64
 9   Fan_5            155 non-null    float64
 10  Fan_6            149 non-null    float64
 11  Fan_7            161 non-null    float64
 12  Fan_8            158 non-null    float64
 13  Fan_9            162 non-null    float64
 14  Fan_10           158 non-null    float64
 15  Fan_11           164 non-null    float64
 16  Fan_12        

###ESTO ES PARA ARREGLAR LAS COLUMNAS###

In [20]:
#I DID THIS WHEN I WAS LEARNING WHAT I HAD TO DO 
#IT DOESN'T MAKE ANY DIFFERENCE ON THE CODE NOW BUT I HAD TO DO IT FIRST BY HAND BEFORE DOING IT IN LOOPS 
#team4['Fan_14']=team4['Fan_14'].replace('NA', np.nan)
#team4['Fan_14']=team4['Fan_14'].replace('NS', np.nan)
#team4['Fan_14']=team4['Fan_14'].str.replace(',', '')
#team4['Fan_14']=team4['Fan_14'].astype(float)

print(np.array(team4['Fan_14']))




[ 99.  60.  92.  83.  86.  78.  67.  99.  82.  86.  75.  76.  66.  88.
  80.  67.  94.  85.  74.  61.  73.  67.  91.  92.  70.  90.  70.  71.
  91.  69.  68.  72.  77.  73.  79.  61.  73.  86.  69.  95.  94.  71.
  82.  73.  89.  66.  77.  91.  87.  75.   7.   7.  nan   8.   9.   7.
   7.  75.  nan  nan   8.  nan   8.  nan   8.  95.  65.   8.   7.  nan
   7.   7.   6.   7.   8.   6.  85.  75.   8.  nan   7.   7.   7.  nan
   8.   6.  nan   9.  75.  nan  nan   8.  75.  85.   8.  85.  nan   8.
   7.  nan  37.  22.  87.  52.  47.  67.  94.  94.  22.  27.  57.  70.
  65.  79.  87.  17.  45.  78.  94.  73.  57.  32.  47.  97.  99.  nan
  37.  nan  27.  72.  12.  17.  43.  nan  92.  37.  77.  82.  70.  60.
  37.  27.  47.  86.  87.  nan  97.  94.  76.  36.  nan  nan  97.  88.
  91.  nan  97.  nan  86.  88.  90.  94.  nan  nan  nan  nan  84.  nan
  nan  66.  85.  nan  nan  98.  nan  68.  76.  nan  70.  99.  nan  nan
  nan  nan  65.  73. 100.  69.  99.  89.  69.  74.  82.  nan  91.  94.
  97. 

In [21]:
team4['Fan_14']=team4['Fan_14'].astype(float)

print(np.array(team4['Fan_14']))

 


[ 99.  60.  92.  83.  86.  78.  67.  99.  82.  86.  75.  76.  66.  88.
  80.  67.  94.  85.  74.  61.  73.  67.  91.  92.  70.  90.  70.  71.
  91.  69.  68.  72.  77.  73.  79.  61.  73.  86.  69.  95.  94.  71.
  82.  73.  89.  66.  77.  91.  87.  75.   7.   7.  nan   8.   9.   7.
   7.  75.  nan  nan   8.  nan   8.  nan   8.  95.  65.   8.   7.  nan
   7.   7.   6.   7.   8.   6.  85.  75.   8.  nan   7.   7.   7.  nan
   8.   6.  nan   9.  75.  nan  nan   8.  75.  85.   8.  85.  nan   8.
   7.  nan  37.  22.  87.  52.  47.  67.  94.  94.  22.  27.  57.  70.
  65.  79.  87.  17.  45.  78.  94.  73.  57.  32.  47.  97.  99.  nan
  37.  nan  27.  72.  12.  17.  43.  nan  92.  37.  77.  82.  70.  60.
  37.  27.  47.  86.  87.  nan  97.  94.  76.  36.  nan  nan  97.  88.
  91.  nan  97.  nan  86.  88.  90.  94.  nan  nan  nan  nan  84.  nan
  nan  66.  85.  nan  nan  98.  nan  68.  76.  nan  70.  99.  nan  nan
  nan  nan  65.  73. 100.  69.  99.  89.  69.  74.  82.  nan  91.  94.
  97. 

In [22]:
# I changed by hand everything in fan_14 to change the scale 
team4['Fan_14'][50:100]
team4['Fan_14'][50:57]=team4['Fan_14'][50:57]*10
team4['Fan_14'][50:57]
team4['Fan_14'][58:65]=team4['Fan_14'][58:65]*10
team4['Fan_14'][58:65]

team4['Fan_14'][67:74]=team4['Fan_14'][67:74]*10
team4['Fan_14'][67:74]
team4['Fan_14'][50:100]
team4['Fan_14'][74:100]
team4['Fan_14'][74:76]=team4['Fan_14'][74:76]*10
team4['Fan_14'][74:76]
team4['Fan_14'][78:88]=team4['Fan_14'][78:88]*10
team4['Fan_14'][78:88]
team4['Fan_14'][74:100]
team4['Fan_14'][96:100]=team4['Fan_14'][96:100]*10
team4['Fan_14'][96:100]
team4['Fan_14'][94]=team4['Fan_14'][94]*10
team4['Fan_14'][91]=team4['Fan_14'][91]*10

C:\Users\jorge\AppData\Local\Temp\ipykernel_33760\767055079.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team4['Fan_14'][50:57]=team4['Fan_14'][50:57]*10
C:\Users\jorge\AppData\Local\Temp\ipykernel_33760\767055079.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team4['Fan_14'][58:65]=team4['Fan_14'][58:65]*10
C:\Users\jorge\AppData\Local\Temp\ipykernel_33760\767055079.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team4

In [23]:


#This is a for loop to do what i previosly did by hand 
#i know that the values with different scale are from 50-100 
#I check the first 100 values and multiply them by 10 if they are smaller than 10 

rango = 199
#I first did it with a clumn to check that my solution was correct 
#for i in range(rango):
   # if team4[seleccionada].iloc[i] < 10:
       # print(team4[seleccionada].iloc[i])
        #team4[seleccionada].iloc[i]=team4[seleccionada].iloc[i] *10
#####################################################################

#It did work on the colum so i did another for loop to do it in all the columns 
####################################################################3
seleccionadas = team4.columns[5:25]

for i in seleccionadas:
    for j in range(rango):
        if team4[i].iloc[j] < 10:
            print(team4[i].iloc[j])
            team4[i].iloc[j] = team4[i].iloc[j] * 10


8.0
9.0
8.0
8.0
8.0
7.0
4.0
9.0
7.0
9.0
8.0
9.0
7.0
8.0
7.0
9.0
9.0
7.0
9.0
8.0
7.0
8.0
8.0
5.0
7.0
9.0
7.0
9.0
9.0
9.0
9.0
8.0
9.0
8.0
9.0
6.0
8.0
8.0
9.0
8.0
8.0
8.0
7.0
8.0
7.0
8.0
8.0
8.0
8.0
8.0
8.0
9.0
7.0
9.0
8.0
8.0
9.0
6.0
6.0
9.0
8.0
8.0
9.0
9.0
9.0
9.0
8.0
8.0
9.0
6.0
9.0
7.0
9.0
9.0
8.0
7.0
7.0
7.0
9.0
7.0
9.0
7.0
9.0
9.0
8.0
8.0
9.0
6.0
7.0
6.0
9.0
8.0
9.0
9.0
9.0
7.0
7.0
9.0
7.0
8.0
9.0
7.0
7.0
7.0
9.0
8.0
8.0
8.0
9.0
9.0
8.0
7.0
8.0
9.0
6.0
8.0
7.0
9.0
8.0
9.0
8.0
6.0
6.0
6.0
9.0
7.0
7.0
8.0
7.0
7.0
7.0
9.0
7.0
9.0
8.0
9.0
8.0
8.0
9.0
6.0
8.0
9.0
7.0
7.0
7.0
9.0
8.0
7.0
7.0
7.0
9.0
7.0
6.0
7.0
7.0
7.0
7.0
6.0
7.0
8.0
8.0
7.0
7.0
7.0
8.0
8.0
8.0
8.0
9.0
8.0
8.0
8.0
8.0
9.0
7.0
7.0
7.0
9.0
7.0
9.0
9.0
7.0
7.0
7.0
8.0
7.0
8.0
7.0
8.0
8.0
7.0
9.0
8.0
8.0
7.0
7.0
9.0
7.0
7.0
9.0
8.0
8.0
8.0
8.0
9.0
7.0
7.0
8.0
6.0
8.0
9.0
9.0
7.0
7.0
8.0
8.0
7.0
8.0
7.0
8.0
8.0
7.0
8.0
7.0
9.0
7.0
8.0
9.0
7.0
9.0
9.0
8.0
8.0
6.0
8.0
8.0
8.0
7.0
7.0
9.0
8.0
8.0
8.0
8.0
8.0
7.0
8.0
7.0
7.0
8.0


C:\Users\jorge\AppData\Local\Temp\ipykernel_33760\2389593242.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team4[i].iloc[j] = team4[i].iloc[j] * 10
C:\Users\jorge\AppData\Local\Temp\ipykernel_33760\2389593242.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team4[i].iloc[j] = team4[i].iloc[j] * 10
C:\Users\jorge\AppData\Local\Temp\ipykernel_33760\2389593242.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team4[i].iloc[j

In [24]:
#Verify that there are not any 1-10 values left 

seleccionada = team4.columns[7]
rango = 100
for i in range(rango):
    if team4[seleccionada].iloc[i] < 10:
        print(team4[seleccionada].iloc[i])



In [25]:
print(np.array(team4['Fan_1']))


[ 90.  78.  97.  72.  90.  76.  76.  72.  87.  69.  82.  96.  87.  84.
  97.  93.  89.  88.  77.  69.  65.  89.  63.  76.  83.  94.  84.  87.
  87.  67.  98.  86.  67.  80.  88.  85.  96.  92.  99.  91.  89.  67.
  94.  86.  91.  89.  69.  71.  80.  77.  nan  80.  nan  nan  nan  65.
  90.  80.  85.  nan  80.  nan  nan  95.  nan  80.  70.  10.  nan  40.
  85.  nan  nan  75.  90.  70.  90.  nan  80.  nan  85.  nan  90.  nan
  85.  65.  nan  70.  80.  70.  90.  90.  70.  90.  80.  70.  85.  75.
  80.  80.  30.  15.  80.  45.  40.  60.  87.  87.  15.  20.  50.  63.
  58.  72.  80.  10.  38.  71.  87.  nan  nan  25.  40.  90.  92.  98.
  30.  97.  20.  65.  50.  10.  36.  95.  85.  30.  70.  75.  63.  53.
  30.  20.  40.  79.  80.  nan  90.  87.  69.  29.  85.  nan  nan  nan
  82.  nan  95.  nan  nan  nan  nan  nan  nan  81.  nan  nan  nan  96.
  nan  98. 100.  89.  nan  nan  91.  94.  97.  nan  88.  nan  nan  nan
  nan  82.  nan  86.  80. 100.  nan  87.  nan  nan  91.  97.  83.  nan
  84. 

In [26]:
#We do this to check that the values make sense 
team4.iloc[:,0:26].describe()

,Year,Duration,Budget,Revenue,Fan_1,Fan_2,Fan_3,Fan_4,Fan_5,Fan_6,...,Fan_11,Fan_12,Fan_13,Fan_14,Fan_15,Fan_16,Fan_17,Fan_18,Fan_19,Fan_20
count,200.000000,200.000000,195.000000,198.000000,153.000000,158.000000,153.000000,145.000000,155.000000,149.000000,...,164.000000,158.000000,160.000000,161.000000,160.000000,145.000000,155.000000,151.000000,160.000000,163.000000
mean,1995.445000,130.755000,93.190052,296.220540,74.568627,81.094937,80.555556,82.455172,80.238710,81.375839,...,80.573171,78.594937,80.693750,73.745342,80.931250,81.786207,81.722581,78.980132,78.043750,78.619632
std,22.168994,28.905581,101.880382,467.318229,21.899521,11.533826,12.643851,11.312468,10.878362,11.220443,...,12.043918,12.135140,10.986554,18.892220,10.441886,9.434118,9.437790,11.673029,13.626347,12.909566
min,1902.000000,14.000000,0.200000,0.041960,10.000000,10.000000,10.000000,60.000000,60.000000,35.000000,...,10.000000,10.000000,40.000000,12.000000,60.000000,60.000000,50.000000,44.000000,10.000000,41.000000
25%,1982.750000,114.500000,13.046000,1.709625,69.000000,75.000000,70.000000,72.000000,72.000000,71.000000,...,72.000000,70.000000,72.000000,68.000000,73.750000,75.000000,76.000000,70.000000,70.000000,70.000000
50%,2000.000000,129.000000,55.000000,90.891504,80.000000,82.000000,82.000000,85.000000,80.000000,84.000000,...,80.000000,80.000000,80.000000,76.000000,81.500000,82.000000,82.000000,80.000000,80.000000,80.000000
75%,2012.250000,147.250000,150.000000,438.387644,89.000000,90.000000,90.000000,90.000000,90.000000,90.000000,...,90.000000,87.750000,88.000000,87.000000,90.000000,90.000000,89.500000,89.000000,87.000000,89.000000
max,2023.000000,218.000000,534.858444,2924.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000


In [27]:
team4['Genre']

Title
The Shawshank Redemption              Drama
The Godfather                         Crime
The Dark Knight                      Action
The Godfather Part II                 Crime
12 Angry Men                          Crime
                                      ...  
Blade Runnner 2049                   Action
Avatar                               Action
Scott Pilgrim vs the world           Action
Psycho                               Horror
Everything Everywhere All at Once    Action
Name: Genre, Length: 200, dtype: object

In [28]:
#This is a for loop to eliminate the additional genres in each movie ( we only need one) 
for i, v in team4.Genre.items():
    v=re.split('/|,', v)[0]
    team4.Genre[i]=v
team4['Genre']=team4['Genre'].str.replace(' ', '')

C:\Users\jorge\AppData\Local\Temp\ipykernel_33760\2993088894.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team4.Genre[i]=v
C:\Users\jorge\AppData\Local\Temp\ipykernel_33760\2993088894.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team4.Genre[i]=v
C:\Users\jorge\AppData\Local\Temp\ipykernel_33760\2993088894.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team4.Genre[i]=v
C:\Users\jorge\AppData\Local\Temp\ipykernel_33760

In [29]:
team4['Fan_3']


Title
The Shawshank Redemption             86.0
The Godfather                        73.0
The Dark Knight                      66.0
The Godfather Part II                70.0
12 Angry Men                         68.0
                                     ... 
Blade Runnner 2049                   99.0
Avatar                               80.0
Scott Pilgrim vs the world           82.0
Psycho                               84.0
Everything Everywhere All at Once     NaN
Name: Fan_3, Length: 200, dtype: float64

In [30]:
team4

,Year,Genre,Duration,Budget,Revenue,Fan_1,Fan_2,Fan_3,Fan_4,Fan_5,...,Fan_16,Fan_17,Fan_18,Fan_19,Fan_20,IMBD,Rotten_Tomatoes,Metacritic,Letterboxd,Personal
Title,,,,,,,,,,,,,,,,,,,,,
The Shawshank Redemption,1994,Drama,142,28.341469,2.343110,90.0,75.0,86.0,61.0,74.0,...,87.0,93.0,70.0,76.0,62.0,93,99,98.0,84,80.0
The Godfather,1972,Crime,175,134.966411,1.620367,78.0,98.0,73.0,71.0,85.0,...,85.0,60.0,98.0,76.0,96.0,92,80,97.0,91,100.0
The Dark Knight,2008,Action,152,534.858444,2.303232,97.0,98.0,66.0,82.0,82.0,...,69.0,76.0,97.0,64.0,74.0,90,95,74.0,70,84.0
The Godfather Part II,1974,Crime,202,57.300000,1.129952,72.0,78.0,70.0,68.0,95.0,...,63.0,87.0,61.0,83.0,75.0,90,82,75.0,91,90.0
12 Angry Men,1957,Crime,96,4.360000,0.689845,90.0,76.0,68.0,96.0,79.0,...,71.0,89.0,89.0,90.0,67.0,90,81,80.0,86,96.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Blade Runnner 2049,2017,Action,164,150.000000,260.500000,NaN,94.0,99.0,NaN,66.0,...,80.0,68.0,80.0,86.0,93.0,80,88,81.0,80,90.0
Avatar,2009,Action,162,237.000000,2924.000000,84.0,97.0,80.0,67.0,67.0,...,NaN,76.0,96.0,NaN,94.0,78,82,83.0,78,70.0
Scott Pilgrim vs the world,2010,Action,112,85.000000,48.100000,NaN,83.0,82.0,68.0,NaN,...,NaN,96.0,NaN,NaN,65.0,75,82,69.0,76,84.0


In [31]:
team4.info()

<class 'pandas.core.frame.DataFrame'>
Index: 200 entries, The Shawshank Redemption to Everything Everywhere All at Once
Data columns (total 30 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Year             200 non-null    int64  
 1   Genre            200 non-null    object 
 2   Duration         200 non-null    int64  
 3   Budget           195 non-null    float64
 4   Revenue          198 non-null    float64
 5   Fan_1            153 non-null    float64
 6   Fan_2            158 non-null    float64
 7   Fan_3            153 non-null    float64
 8   Fan_4            145 non-null    float64
 9   Fan_5            155 non-null    float64
 10  Fan_6            149 non-null    float64
 11  Fan_7            161 non-null    float64
 12  Fan_8            158 non-null    float64
 13  Fan_9            162 non-null    float64
 14  Fan_10           158 non-null    float64
 15  Fan_11           164 non-null    float64
 16  Fan_12        

In [32]:
#Change the index column to string 
team4.index = team4.index.astype(str)


# Capitalize the first letter of each value in the Title column
team4.index = team4.index.str.capitalize()


In [33]:
team4.info()

<class 'pandas.core.frame.DataFrame'>
Index: 200 entries, The shawshank redemption to Everything everywhere all at once
Data columns (total 30 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Year             200 non-null    int64  
 1   Genre            200 non-null    object 
 2   Duration         200 non-null    int64  
 3   Budget           195 non-null    float64
 4   Revenue          198 non-null    float64
 5   Fan_1            153 non-null    float64
 6   Fan_2            158 non-null    float64
 7   Fan_3            153 non-null    float64
 8   Fan_4            145 non-null    float64
 9   Fan_5            155 non-null    float64
 10  Fan_6            149 non-null    float64
 11  Fan_7            161 non-null    float64
 12  Fan_8            158 non-null    float64
 13  Fan_9            162 non-null    float64
 14  Fan_10           158 non-null    float64
 15  Fan_11           164 non-null    float64
 16  Fan_12        

## Merging databases

In [34]:

team4

,Year,Genre,Duration,Budget,Revenue,Fan_1,Fan_2,Fan_3,Fan_4,Fan_5,...,Fan_16,Fan_17,Fan_18,Fan_19,Fan_20,IMBD,Rotten_Tomatoes,Metacritic,Letterboxd,Personal
Title,,,,,,,,,,,,,,,,,,,,,
The shawshank redemption,1994,Drama,142,28.341469,2.343110,90.0,75.0,86.0,61.0,74.0,...,87.0,93.0,70.0,76.0,62.0,93,99,98.0,84,80.0
The godfather,1972,Crime,175,134.966411,1.620367,78.0,98.0,73.0,71.0,85.0,...,85.0,60.0,98.0,76.0,96.0,92,80,97.0,91,100.0
The dark knight,2008,Action,152,534.858444,2.303232,97.0,98.0,66.0,82.0,82.0,...,69.0,76.0,97.0,64.0,74.0,90,95,74.0,70,84.0
The godfather part ii,1974,Crime,202,57.300000,1.129952,72.0,78.0,70.0,68.0,95.0,...,63.0,87.0,61.0,83.0,75.0,90,82,75.0,91,90.0
12 angry men,1957,Crime,96,4.360000,0.689845,90.0,76.0,68.0,96.0,79.0,...,71.0,89.0,89.0,90.0,67.0,90,81,80.0,86,96.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Blade runnner 2049,2017,Action,164,150.000000,260.500000,NaN,94.0,99.0,NaN,66.0,...,80.0,68.0,80.0,86.0,93.0,80,88,81.0,80,90.0
Avatar,2009,Action,162,237.000000,2924.000000,84.0,97.0,80.0,67.0,67.0,...,NaN,76.0,96.0,NaN,94.0,78,82,83.0,78,70.0
Scott pilgrim vs the world,2010,Action,112,85.000000,48.100000,NaN,83.0,82.0,68.0,NaN,...,NaN,96.0,NaN,NaN,65.0,75,82,69.0,76,84.0


In [35]:
#movies_df.shape

## Checking for duplicates

In [36]:
duplicate_titles=team4[team4.index.duplicated()].index
team4.loc[duplicate_titles]
#In this line we check for duploicates and add them into a list 

,Year,Genre,Duration,Budget,Revenue,Fan_1,Fan_2,Fan_3,Fan_4,Fan_5,...,Fan_16,Fan_17,Fan_18,Fan_19,Fan_20,IMBD,Rotten_Tomatoes,Metacritic,Letterboxd,Personal
Title,,,,,,,,,,,,,,,,,,,,,
The usual suspects,1995,Crime,106,23.341568,0.991208,67.0,75.0,87.0,79.0,72.0,...,76.0,84.0,76.0,69.0,70.0,95,95,71.0,75,77.0
The usual suspects,1995,Crime,106,6.000000,23.341568,60.0,80.0,75.0,NaN,70.0,...,NaN,85.0,79.0,NaN,72.0,85,88,76.0,"4,1",NaN
Goodfellas,1990,Biography,146,46.836394,1.020727,93.0,83.0,67.0,91.0,62.0,...,68.0,71.0,92.0,63.0,79.0,96,84,75.0,70,90.0
Goodfellas,1990,Biography,145,25.000000,47.036784,87.0,95.0,90.0,NaN,92.0,...,75.0,90.0,94.0,95.0,91.0,87,95,92.0,"4,5",93.0
Saving private ryan,1998,Drama,169,216.540909,1.235804,83.0,93.0,90.0,95.0,99.0,...,88.0,86.0,77.0,70.0,100.0,97,93,87.0,78,91.0
Saving private ryan,1998,Drama,169,70.000000,482.349603,40.0,85.0,90.0,84.0,80.0,...,85.0,88.0,78.0,79.0,70.0,86,94,91.0,"4,2",83.0
Top gun maverick,2022,Action,130,170.000000,1495.696292,90.0,NaN,NaN,NaN,60.0,...,NaN,NaN,NaN,80.0,80.0,82,96,78.0,80,NaN
Top gun maverick,2022,Action,144,152.000000,1496.000000,NaN,89.0,94.0,96.0,96.0,...,91.0,99.0,82.0,100.0,70.0,82,96,78.0,80,75.0
Joker,2019,Crime,122,335.451311,0.939252,80.0,72.0,79.0,89.0,94.0,...,83.0,75.0,73.0,86.0,90.0,79,86,89.0,86,59.0


In [37]:
team4 = team4[~team4.index.duplicated(keep='first')]
team4.loc[duplicate_titles]
#We remove the duplicates and only keep the first one 

,Year,Genre,Duration,Budget,Revenue,Fan_1,Fan_2,Fan_3,Fan_4,Fan_5,...,Fan_16,Fan_17,Fan_18,Fan_19,Fan_20,IMBD,Rotten_Tomatoes,Metacritic,Letterboxd,Personal
Title,,,,,,,,,,,,,,,,,,,,,
The usual suspects,1995,Crime,106,23.341568,0.991208,67.0,75.0,87.0,79.0,72.0,...,76.0,84.0,76.0,69.0,70.0,95,95,71.0,75,77.0
Goodfellas,1990,Biography,146,46.836394,1.020727,93.0,83.0,67.0,91.0,62.0,...,68.0,71.0,92.0,63.0,79.0,96,84,75.0,70,90.0
Saving private ryan,1998,Drama,169,216.540909,1.235804,83.0,93.0,90.0,95.0,99.0,...,88.0,86.0,77.0,70.0,100.0,97,93,87.0,78,91.0
Top gun maverick,2022,Action,130,170.000000,1495.696292,90.0,NaN,NaN,NaN,60.0,...,NaN,NaN,NaN,80.0,80.0,82,96,78.0,80,NaN
Joker,2019,Crime,122,335.451311,0.939252,80.0,72.0,79.0,89.0,94.0,...,83.0,75.0,73.0,86.0,90.0,79,86,89.0,86,59.0
Psycho,1960,Horror,109,0.522093,1.024560,77.0,64.0,67.0,72.0,80.0,...,83.0,81.0,61.0,87.0,93.0,90,96,88.0,71,NaN


In [38]:
team4.shape
team4

,Year,Genre,Duration,Budget,Revenue,Fan_1,Fan_2,Fan_3,Fan_4,Fan_5,...,Fan_16,Fan_17,Fan_18,Fan_19,Fan_20,IMBD,Rotten_Tomatoes,Metacritic,Letterboxd,Personal
Title,,,,,,,,,,,,,,,,,,,,,
The shawshank redemption,1994,Drama,142,28.341469,2.343110,90.0,75.0,86.0,61.0,74.0,...,87.0,93.0,70.0,76.0,62.0,93,99,98.0,84,80.0
The godfather,1972,Crime,175,134.966411,1.620367,78.0,98.0,73.0,71.0,85.0,...,85.0,60.0,98.0,76.0,96.0,92,80,97.0,91,100.0
The dark knight,2008,Action,152,534.858444,2.303232,97.0,98.0,66.0,82.0,82.0,...,69.0,76.0,97.0,64.0,74.0,90,95,74.0,70,84.0
The godfather part ii,1974,Crime,202,57.300000,1.129952,72.0,78.0,70.0,68.0,95.0,...,63.0,87.0,61.0,83.0,75.0,90,82,75.0,91,90.0
12 angry men,1957,Crime,96,4.360000,0.689845,90.0,76.0,68.0,96.0,79.0,...,71.0,89.0,89.0,90.0,67.0,90,81,80.0,86,96.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Et,1982,Family,115,10.500000,792.900000,83.0,91.0,NaN,NaN,NaN,...,NaN,90.0,69.0,83.0,NaN,78,98,91.0,80,75.0
Blade runnner 2049,2017,Action,164,150.000000,260.500000,NaN,94.0,99.0,NaN,66.0,...,80.0,68.0,80.0,86.0,93.0,80,88,81.0,80,90.0
Avatar,2009,Action,162,237.000000,2924.000000,84.0,97.0,80.0,67.0,67.0,...,NaN,76.0,96.0,NaN,94.0,78,82,83.0,78,70.0


In [39]:
#Save the new and clean dataset into a file
team4.to_csv('movie_DB_clean.csv')